# Robustness Check: Remove General Terms from Top 100 Noun Chunks in Each Area

In [1]:
import pandas as pd
import os
import datetime
import pickle
import re
import time
from collections import Counter
import numpy as np

import nltk
nltk.data.path

['/home/ec2-user/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/share/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/lib/nltk_data',
 '/usr/share/nltk_data',
 '/usr/local/share/nltk_data',
 '/usr/lib/nltk_data',
 '/usr/local/lib/nltk_data']

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [4]:
from ast import literal_eval
from collections import Counter

In [5]:
import statsmodels.formula.api as sm

## 1. Import Regulatory Sections and Noun Chunks with Areas

In [6]:
# Noun chunks with areas
nounchunks_area=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/DictionaryOfRegulatoryNounChunks.csv')
print(nounchunks_area.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10645 entries, 0 to 10644
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nc_code      10645 non-null  int64 
 1   noun_chunks  10645 non-null  object
 2   rin          10645 non-null  object
 3   area         10185 non-null  object
 4   area_no      10645 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 415.9+ KB
None


In [7]:
nounchunks_area.head()

,nc_code,noun_chunks,rin,area,area_no
0,0,180-day exclusivity,0910-AC11,{1},1
1,1,1983 amendment,2115-AB72,{2},1
2,2,1988 amendment,1205-AB05,{4},1
3,3,1990 farm bill,0584-AB28,{1},1
4,4,1993 provision,"0970-AB32,3206-AG31",{1},1


In [8]:
# Remove area-specific general terms
nounchunks_remove=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/Top 100 Area-specific Noun Chunks Removed.csv')
print(nounchunks_remove.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1385 entries, 0 to 1384
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   noun_chunks      1385 non-null   object 
 1   area_no          1385 non-null   int64  
 2   area_name        1385 non-null   object 
 3   remove_dda_less  258 non-null    float64
 4   remove_dda_more  449 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 54.2+ KB
None


In [9]:
# Specify which removal appraoch to use
nounchunks_remove=nounchunks_remove.rename(columns={'remove_dda_more':'remove_dda'})
print(nounchunks_remove['remove_dda'].value_counts())

1.0    449
Name: remove_dda, dtype: int64


In [10]:
# Merge
nounchunks_area=nounchunks_area.merge(nounchunks_remove[['noun_chunks','remove_dda']],on='noun_chunks',how='left')
print(nounchunks_area.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10645 entries, 0 to 10644
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   nc_code      10645 non-null  int64  
 1   noun_chunks  10645 non-null  object 
 2   rin          10645 non-null  object 
 3   area         10185 non-null  object 
 4   area_no      10645 non-null  int64  
 5   remove_dda   449 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 582.1+ KB
None


In [11]:
# Convert to dictionary
nounchunks_area=nounchunks_area[(nounchunks_area['area_no']>0) & (nounchunks_area['remove_dda']!=1)].set_index('noun_chunks')
nounchunks_area_dict=nounchunks_area.to_dict()['area']
print(len(nounchunks_area_dict))

9736


In [12]:
# Expanded reg sentences with matched noun chunks
df_regSentsExpand=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_NounChunks3.pkl')
print(df_regSentsExpand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           822737 non-null  object
 1   RegSentsExpand               822737 non-null  object
 2   NounChunksMatch              822737 non-null  int64 
 3   NounChunkMatchWords          822737 non-null  object
 4   NounChunkMatchWordsFiltered  822737 non-null  object
 5   NounChunkMatchFiltered       822737 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 37.7+ MB
None


In [13]:
# Remove duplicated articles
IDs_nodup=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/IDs_no_duplicates.csv')
print(IDs_nodup.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788516 entries, 0 to 788515
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   ID      788516 non-null  int64
dtypes: int64(1)
memory usage: 6.0 MB
None


In [14]:
df_regSentsExpand['ID']=df_regSentsExpand['ID'].astype('int64')
df_regSentsExpand=IDs_nodup.merge(df_regSentsExpand,on='ID',how='left').reset_index(drop=True)
print(df_regSentsExpand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788516 entries, 0 to 788515
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           788516 non-null  int64 
 1   RegSentsExpand               788516 non-null  object
 2   NounChunksMatch              788516 non-null  int64 
 3   NounChunkMatchWords          788516 non-null  object
 4   NounChunkMatchWordsFiltered  788516 non-null  object
 5   NounChunkMatchFiltered       788516 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 36.1+ MB
None


In [15]:
# Refine to reg relevant articles
df_regSentsExpandRelevant=df_regSentsExpand[df_regSentsExpand['NounChunkMatchFiltered']>0].reset_index(drop=True)
print(df_regSentsExpandRelevant.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493418 entries, 0 to 493417
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           493418 non-null  int64 
 1   RegSentsExpand               493418 non-null  object
 2   NounChunksMatch              493418 non-null  int64 
 3   NounChunkMatchWords          493418 non-null  object
 4   NounChunkMatchWordsFiltered  493418 non-null  object
 5   NounChunkMatchFiltered       493418 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 22.6+ MB
None


## 2. Link Expanded Reg Sentences to Areas

### Approach 4: dominant distinct area (dda): use the dominant areas from area-specific noun chunks

In [ ]:
df_regSentsExpandRelevant['AllDistinctAreas']=''
df_regSentsExpandRelevant['DistinctAreaCount']=''
df_regSentsExpandRelevant['DominantDistinctArea']=''
for i in range(0, len(df_regSentsExpandRelevant)):
    nounchunks=df_regSentsExpandRelevant['NounChunkMatchWordsFiltered'][i]
    area_list=[]
    for nc in nounchunks:
        if nc in nounchunks_area_dict:
            area=sorted(literal_eval(nounchunks_area_dict[nc]))
            if len(area)==1:
                area_list=area_list+area    
    
    area_count=Counter(area_list).most_common()
    dominant_area=[j for j in Counter(area_list).keys() if area_list.count(j)==max(Counter(area_list).values())]
    df_regSentsExpandRelevant['AllDistinctAreas'][i]=area_list
    df_regSentsExpandRelevant['DistinctAreaCount'][i]=area_count
    df_regSentsExpandRelevant['DominantDistinctArea'][i]=dominant_area
print(df_regSentsExpandRelevant.info())

/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/ec2-user/SageMaker/.conda/envs/my_py/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [ ]:
print(df_regSentsExpandRelevant.head())

In [ ]:
df_regSentsExpandRelevant.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegSentsExpand_NounChunks_Area_Robust2.pkl')

## 3. Filtered Noun Chunk Occurences by Area

In [ ]:
# Reg relevant articles
df_regSentsExpandRelevant=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegSentsExpand_NounChunks_Area_Robust2.pkl')
print(df_regSentsExpandRelevant.info())

In [ ]:
# Filtered noun chunk occurences across regulation-related articles
df_nounchunk_occurences=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_FilteredNounChunkOccurences.csv')
print(df_nounchunk_occurences.info())

In [ ]:
# Dummies by dominant distinct area (dda)
for i in range(1,15):
    var='DominantDistinctArea'+str(i)
    df_regSentsExpandRelevant[var]=0
    for j in range(0, len(df_regSentsExpandRelevant)):
        if i in df_regSentsExpandRelevant['DominantDistinctArea'][j]:
            df_regSentsExpandRelevant[var][j]=1

In [ ]:
# Filtered noun chunks across regulation-related articles by area
for i in range(1,15):
    allDistinctAreas=[]
    for allArea in df_regSentsExpandRelevant[df_regSentsExpandRelevant['DominantDistinctArea'+str(i)]==1]['NounChunkMatchWordsFiltered']:
        distinctArea=[nc for nc in allArea if nc in nounchunks_area_dict]
        allDistinctAreas=allDistinctAreas+distinctArea
    allDistinctAreaCount=Counter(allDistinctAreas)
    var_name='Occurences_dda'+str(i)
    df_MatchWords = pd.DataFrame(allDistinctAreaCount.items(),columns = ['Noun Chunks',var_name])
    df_nounchunk_occurences=df_nounchunk_occurences.merge(df_MatchWords,on='Noun Chunks',how='outer')
print(df_nounchunk_occurences)

In [ ]:
df_nounchunk_occurences.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_FilteredNounChunkOccurences_Robust2.csv',index=False)

## 4. Aggregate sentiment scores

In [ ]:
# Reg relevant articles
df_regSentsExpandRelevant=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegSentsExpand_NounChunks_Area_Robust2.pkl')
print(df_regSentsExpandRelevant.info())

In [ ]:
# All news data
allNews=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/parsed_xml.pkl')
print(allNews.info())

In [ ]:
# Merge with all news data
allNews['ID']=allNews['ID'].astype('int64')
df=df_regSentsExpandRelevant.merge(allNews[['ID','Title','Type','SourceType','StartDate','Newspaper','Year','Month']],
                                  on='ID',how='left')
print(df.info())

In [ ]:
# List of columns for different approaches
area_range=15    # Number of areas + 1
col_list=[]
for i in range(1,area_range):
    var='DominantDistinctArea'+str(i)
    col_list.append(var)

In [ ]:
# Dummies by dominant distinct area (dda)
for i in range(1,area_range):
    var='DominantDistinctArea'+str(i)
    df[var]=0
    for j in range(0, len(df)):
        if i in df['DominantDistinctArea'][j]:
            df[var][j]=1
#print(df.info())

In [ ]:
# Merge with sentiment scores
sentimentScores=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/RegRelevant_ArticleSentimentScores.csv')
print(sentimentScores.info())

In [ ]:
# Merge
df['ID']=df['ID'].astype('int64')
df2=df.merge(sentimentScores[['ID','UncertaintyScore','GIscore','LMscore','LSDscore']],on='ID',how='left')
#print(df2.info())

In [ ]:
df2[['ID','StartDate','Newspaper','UncertaintyScore','GIscore','LMscore','LSDscore']+col_list].\
    to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_ArticleSentimentScores_Robust2.csv',index=False)

## 5. Monthly article count by area

In [ ]:
# Aggregate monthly article counts
monthlyAreaCount=df[['Newspaper','Year','Month']+col_list].groupby(['Newspaper','Year','Month']).agg('sum').reset_index()
print(monthlyAreaCount.info())

In [ ]:
monthlyAreaCount.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_MonthlyArticleCountByNewspaper_Robust2.csv',index=False)

## 6. Construct categorical sentiment indexes

In [ ]:
df=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_ArticleSentimentScores_Robust2.csv')
print(df.info())

In [ ]:
# Reformat data
df['StartDate']=df['StartDate'].astype('datetime64[ns]')
df['Year']=df['StartDate'].dt.year
df['Month']=df['StartDate'].dt.month
df['Newspaper']=df['Newspaper'].astype('category')
#print(df.info())

In [ ]:
# Create year-month dataframe
df_ym=df[['Year','Month']].drop_duplicates().sort_values(['Year','Month']).reset_index(drop=True).reset_index()
df_ym['YM']=df_ym['index']+1
df_ym['YM']=df_ym['YM'].astype('str')
df_ym=df_ym.drop('index',axis=1)
print(df_ym,'\n',len(df_ym))

In [ ]:
# Merge year-month dataframe
df=df.merge(df_ym[['Year','Month','YM']],on=['Year','Month'],how='left').sort_values(['Year','Month']).reset_index(drop=True)
print(df.info())

In [ ]:
df=df.rename(columns={'UncertaintyScore':'Uncertaintyscore'})
YM_list=df_ym['YM'].tolist()
#print(YM_list)

In [ ]:
# Define a function (suppressing constant) to estimate categorical index
def estimate_categorical_index(score, area):
    df_area=df[df[area]==1].reset_index(drop=True)
    FE_OLS=sm.ols(formula=score + ' ~ 0+C(YM)+C(Newspaper)', data=df_area).fit()
    #print(FE_OLS.summary())

    FE_estimates=pd.DataFrame()
    new_var=score.split('score')[0]+'_'+area
    FE_estimates[new_var]=FE_OLS.params[0:len(df_ym)]
    FE_estimates=FE_estimates.reset_index().rename(columns={'index':'FE'})
    FE_estimates['YM']=FE_estimates['FE'].str.split("[",expand=True)[1].str.split("]",expand=True)[0]
    
    for value in FE_estimates['YM']:
        if value not in YM_list:
            FE_estimates=FE_estimates[FE_estimates['YM']!=value]
    FE_estimates=FE_estimates.drop('FE',axis=1)
    
    return FE_estimates

In [ ]:
# List of columns for different classification approaches
area_range=15
area_list=[]
for i in range(1,area_range):
    var='DominantDistinctArea'+str(i)
    area_list.append(var)

In [ ]:
# Categorical Uncertainty Index
CategoricalUncertaintyIndex=df_ym
for area in area_list:
    try:
        estimates=estimate_categorical_index('Uncertaintyscore', area)
        CategoricalUncertaintyIndex=CategoricalUncertaintyIndex.merge(estimates,on='YM',how='left')
    except:
        print("Failed:",area)

In [ ]:
CategoricalUncertaintyIndex.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_MonthlyUncertaintyIndex_Robust2.csv',index=False)

In [ ]:
# Categorical sentiment indexes
for dict in ['GI','LM','LSD']:
    CategoricalSentimentIndex=df_ym
    for area in area_list:
        try:
            estimates=estimate_categorical_index(dict+'score', area)
            CategoricalSentimentIndex=CategoricalSentimentIndex.merge(estimates,on='YM',how='left')
        except:
            print("Failed:",dict+":"+area)      
    CategoricalSentimentIndex.to_csv('/home/ec2-user/SageMaker/New Uncertainty/Categorical Index/RegArea_Monthly'+dict+'Index_Robust2.csv',index=False)